In [1]:
import geopandas as gpd 
import pyodbc
import config
import os
conn = pyodbc.connect('Driver={SQL Server};'
                     f'Server={config.sql_server};'
                     f'Database={config.sql_databse};'
                     f'UID={config.sql_user};'
                     f'PWD={config.sql_pass};')

TODO:

Chunck insert (MAX 1000 - [SQL Server Doc](https://docs.microsoft.com/en-us/sql/t-sql/queries/table-value-constructor-transact-sql?redirectedfrom=MSDN&view=sql-server-ver15))

multi input Directory/Single File/Dataframe

Try... Except... Finally...

SQL Parameters (CRS, Database, Tablename)

Schema Option

In [10]:
def check_table(dbName, tableName, cursor):
    sql = f""" SELECT TABLE_NAME 
        FROM {dbName}.INFORMATION_SCHEMA.TABLES
        WHERE TABLE_TYPE = 'BASE TABLE' """
    cursor.execute(sql)
    row = cursor.fetchone()
    while row:
        if tableName == row[0]:
            return True
        row = cursor.fetchone()

def build_schema(data):
    df_dt = data.dtypes
    schema = {}
    for idx, row in df_dt.iteritems():
        if row == 'int64':
            dataType = 'int'
        elif row == 'float64':
            dataType = 'float'
        elif row == 'geometry':
            dataType = 'geometry'
        else:
            dataType = 'VarChar(250)'
        schema[idx] = dataType
    return schema


def create_table(data,dbName,tableName,cursor,schema=None):
    if schema is None:
        schema = build_schema(data)
    for key, value in schema.items():
        col_dtype_list.append(f'{key} {value}')
    varCharCols = ','.join(col_dtype_list)
    if check_table(dbName, tableName, cursor)
        createTableString = f"""
            CREATE TABLE {tableName} (
                {tableCols}
            );
        """
        cxn.execute(createTableString)
        cxn.commit()

def insert_byRow(data, cursor, crs):
    cols = data.columns
    geo = cols[-1:][0]
    cols = cols[:-1]
    insertHead = ','.join([str(elem) for elem in cols])
    insertHead = insertHead+', polygon'
    for row in data.iterrows():
            valgeo = row[1]['geometry']
            val = row[1].tolist()
            val = val[:-1]
            val = ','.join(["'"+str(elem).replace("'", "''")+"'" for elem in val])
            insertSQL = f"insert into {tableName} ({insertHead}) values ({val}, geometry::STGeomFromText('{valgeo}', {crs}))"
            cursor.execute(insertSQL)
            cursor.commit()

def insert_byChunck(data, cursor, crs):
    list_df = [gdfCD[i:i+n] for i in range(0,gdfCD.shape[0],1000)]
    for i in list_df:
    insertrow = []
    for idx, row in i.iterrows():
        valgeo = row['geometry']
        val = row.tolist()
        val = val[:-1]
        val = ','.join(["'"+str(elem).replace("'", "''")+"'" for elem in val])
        insertSQL = f"({val}, geometry::STGeomFromText('{valgeo}', {crs}))"
        insertrow.append(insertSQL)
    cols = data.columns
    insertHead = ','.join([str(elem) for elem in cols])
    chuckInsertRows = ','.join()
    insertSQL = f"insert into {tableName} ({insertHead}) values {chuckInsertRows}"
    cursor.execute(insertSQL)
    cursor.commit()

def upload_GISdata(data, dbName, tableName, conn, path=None , crs=4269, **schema):
    cursor = conn.cursor()
    if path:
        os.chdir(path)
    data = data.to_crs(epsg=crs)
    
        
    cxn.close()

In [29]:
gdf = gpd.read_file('tl_2021_us_cd116.zip')
gdfCD = gdf[gdf['STATEFP']=='27']
gdfCD.head()

,STATEFP,CD116FP,GEOID,NAMELSAD,LSAD,CDSESSN,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,27,07,2707,Congressional District 7,C2,116,G5200,N,86581556608,5058864376,+46.5530202,-095.6759048,"POLYGON ((-97.23909 48.96826, -97.23909 48.968..."
1,27,02,2702,Congressional District 2,C2,116,G5200,N,6314201751,241412000,+44.4789680,-092.8530418,"POLYGON ((-93.91131 44.54651, -93.91024 44.548..."
2,27,06,2706,Congressional District 6,C2,116,G5200,N,7460842730,409026974,+45.3368069,-093.8520551,"POLYGON ((-94.64041 45.37072, -94.64039 45.371..."
3,27,08,2708,Congressional District 8,C2,116,G5200,N,72283254582,12454422188,+47.2503576,-092.9640512,"POLYGON ((-95.18322 47.41277, -95.16598 47.412..."
51,27,01,2701,Congressional District 1,C2,116,G5200,N,31013752203,540214315,+43.8989394,-093.7117418,"POLYGON ((-96.45347 43.60894, -96.45347 43.609..."


["('27','07','2707','Congressional District 7','C2','116','G5200','N','86581556608','5058864376','+46.5530202','-095.6759048', geometry::STGeomFromText('', 4269))", "('27','02','2702','Congressional District 2','C2','116','G5200','N','6314201751','241412000','+44.4789680','-092.8530418', geometry::STGeomFromText('', 4269))"]
["('27','06','2706','Congressional District 6','C2','116','G5200','N','7460842730','409026974','+45.3368069','-093.8520551', geometry::STGeomFromText('', 4269))", "('27','08','2708','Congressional District 8','C2','116','G5200','N','72283254582','12454422188','+47.2503576','-092.9640512', geometry::STGeomFromText('', 4269))"]
["('27','01','2701','Congressional District 1','C2','116','G5200','N','31013752203','540214315','+43.8989394','-093.7117418', geometry::STGeomFromText('', 4269))", "('27','03','2703','Congressional District 3','C2','116','G5200','N','1366025400','144548321','+44.9959261','-093.5288022', geometry::STGeomFromText('', 4269))"]
["('27','04','2704'